<a href="https://colab.research.google.com/github/DirkStulgies/mlwtSportsPrediction/blob/main/playground/own_data_preparation_dirk.ipynb"
 target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import packages.
import os
import pandas as pd

In [ ]:
# Define parameters.
RESULT_COLUMN = 'result_team1'
SCORE_TEAM1_COLUMN = 'score1'
SCORE_TEAM2_COLUMN = 'score2'
VALUE_HOME_LOST = 0
VALUE_HOME_DRAW = 1
VALUE_HOME_WON = 2

In [ ]:
# Load the baseline data set.
five_projects = pd.read_csv('../data/five_projects_soccer_match_data_modified.csv', delimiter=',', decimal='.') 
five_projects = five_projects.loc[:, ['season', 'date', 'team1', 'team2', 'score1', 'score2']]

five_projects.sort_values(by=['date'], ascending=True, inplace=True)
teams = five_projects['team1'].unique()
seasons = five_projects['season'].unique()

# Add the result.
five_projects[RESULT_COLUMN] = VALUE_HOME_DRAW
five_projects.loc[five_projects[SCORE_TEAM1_COLUMN] < five_projects[SCORE_TEAM2_COLUMN], RESULT_COLUMN] = VALUE_HOME_LOST
five_projects.loc[five_projects[SCORE_TEAM1_COLUMN] > five_projects[SCORE_TEAM2_COLUMN], RESULT_COLUMN] = VALUE_HOME_WON

# Count the matches and calculate the points.
new_columns = pd.DataFrame(data={
    'date': [],
    'match_day_team1': [],
    'match_day_team2': [],
    'points_team1': [],
    'points_team2': []
})
matches = {}
points = {}
for team in teams:
    for season in seasons:
        matches[team + '_' + str(season)] = 0
        points[team + '_' + str(season)] = 0

for idx, row in five_projects.iterrows():
    sea = str(row['season'])
    res = row[RESULT_COLUMN]

    match1 = matches[row['team1']+'_'+sea] + 1
    matches[row['team1']+'_'+sea] = match1

    match2 = matches[row['team2']+'_'+sea] + 1
    matches[row['team2']+'_'+sea] = match2

    if res == 0:
        points[row['team2']+'_'+sea] += 3
    elif res == 1:
        points[row['team1']+'_'+sea] += 1
        points[row['team2']+'_'+sea] += 1
    elif res == 2:
        points[row['team1']+'_'+sea] += 3

    new_columns = new_columns.append(
        {
        'date': row['date'],
        'match_day_team1': match1,
        'match_day_team2': match2,
        'points_team1': points[row['team1']+'_'+sea],
        'points_team2': points[row['team2']+'_'+sea] 
        }, ignore_index=True)

a = pd.merge(five_projects, new_columns, on='date')

# Show data.
a[0:24]

In [ ]:
# Add market values.
market = pd.read_csv('../data/market_value_soccer_german_federal_league.txt', delimiter='\t', decimal='.')
b = pd.merge(a, market, left_on=['season', 'team1'], right_on=['season', 'club'])
c = pd.merge(b, market, left_on=['season', 'team2'], right_on=['season', 'club'], suffixes=('_team1', '_team2'))

In [ ]:
# Add points and ranks.
points = pd.read_csv('../data/points_soccer_german_federal_league.txt', delimiter='\t', decimal='.')
points['season'] = points['season'] + 1
points.rename(columns={'points': 'points_last_season_all', 'rank': 'rank_last_season'}, inplace=True)
d = pd.merge(c, points, left_on=['season', 'team1'], right_on=['season', 'club'])
e = pd.merge(d, points, left_on=['season', 'team2'], right_on=['season', 'club'], suffixes=('_team1', '_team2'))
e['points_last_season_team1'] = e['points_last_season_all_team1'] / e['match_day_team1']
e['points_last_season_team2'] = e['points_last_season_all_team2'] / e['match_day_team2']
e[0:24]

In [ ]:
# Drop same columns and replace empty space in column names.
e.drop(['club_team1', 'club_team2', 'foreigners_team2', 'foreigners_team1'], axis=1, inplace=True)
e.columns = e.columns.str.replace(' ', '_')

In [ ]:
# Show data
e.head()

In [ ]:
# Show data.
e.tail()

In [ ]:
# Write data to file.
e.to_csv('../data/own_data_preparation_dirk.csv', index=False)